# IMPORTAR LIBRERIAS

In [1]:
import mysql.connector as con
import pandas as pd
import pymysql # Para ver la versión
import sqlalchemy # Para ver la versión
from sqlalchemy import create_engine

In [2]:
print(f"Conector: {con.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"pymysql: {pymysql.__version__}")
print(f"sqlalchemy: {sqlalchemy.__version__}")

Conector: 9.1.0
Pandas: 2.2.3
pymysql: 1.4.6
sqlalchemy: 2.0.37


# PARTE 1: CREAR BASE DE DATOS Y CREAR TABLAS

Código SQL para la creacion de base de datos y tablas

In [3]:
sql_creacion = """ 
DROP DATABASE IF EXISTS supermercado;

CREATE DATABASE IF NOT EXISTS supermercado;

USE supermercado;

CREATE TABLE IF NOT EXISTS tiendas (
    id_tienda INT AUTO_INCREMENT PRIMARY KEY,
    nombre_tienda VARCHAR(100),
    direccion VARCHAR(255),
    ciudad VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS empleados (
    id_empleado INT AUTO_INCREMENT PRIMARY KEY,
    nombre_empleado VARCHAR(100),
    puesto VARCHAR(50),
    id_tienda INT NOT NULL,
    FOREIGN KEY (id_tienda) REFERENCES tiendas (id_tienda)
);

CREATE TABLE IF NOT EXISTS categorias (
    id_categoria INT AUTO_INCREMENT PRIMARY KEY,
    nombre_categoria VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS productos (
    id_producto INT AUTO_INCREMENT PRIMARY KEY,
    nombre_producto VARCHAR(100),
    precio DECIMAL(5, 2),
    stock INT,
    id_categoria INT NOT NULL,
    FOREIGN KEY (id_categoria) REFERENCES categorias (id_categoria)
);

CREATE TABLE IF NOT EXISTS clientes (
    id_cliente INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(255),
    codigo_postal INT(5)
);

CREATE TABLE IF NOT EXISTS ordenes (
    id_orden INT AUTO_INCREMENT PRIMARY KEY,
    id_cliente INT NOT NULL,
    id_empleado INT NOT NULL,
    fecha_orden DATETIME,
    metodo_pago ENUM('Tarjeta', 'Efectivo'),
    FOREIGN KEY (id_cliente) REFERENCES clientes (id_cliente),
    FOREIGN KEY (id_empleado) REFERENCES empleados (id_empleado)
);

CREATE TABLE IF NOT EXISTS detalle_orden (
    id_detalle INT AUTO_INCREMENT PRIMARY KEY,
    id_orden INT NOT NULL,
    id_producto INT NOT NULL,
    cantidad INT,
    precio_unitario DECIMAL(5, 2), -- mismo precio que en la tabla producto
    descuento DECIMAL(3, 2),
    FOREIGN KEY (id_orden) REFERENCES ordenes (id_orden),
    FOREIGN KEY (id_producto) REFERENCES productos (id_producto)
);
"""

Conexión a MySQL y ejecución de codigo de creacion

In [4]:
connection = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin"
)

cursor = connection.cursor()
cursor.execute(sql_creacion)
cursor.close()
connection.close()

# PARTE 2: GENERAR DATOS DEMO DESDE PYTHON

In [5]:
import random

## TIENDAS

In [6]:
ciudades = ['Málaga', 'Madrid', 'San Sebastian']
brand = 'SUPER24H'
tipo_via = ['Plaza', 'Calle', 'Avenida']
nombres_calle = ['Costitución', 'Libertad', 'Conocimiento', 'Pacífico', 'América']
tienda = []
for ciudad in ciudades:
    for num in range(1,6):
        direccion = f"{random.choice(tipo_via)}, {random.choice(nombres_calle)}, {random.randint(1, 150)}"
        tienda.append([f'{brand}_{ciudad}_{num}', direccion, ciudad])

df_tienda = pd.DataFrame(tienda, columns=['nombre_tienda', 'direccion', 'ciudad'])
df_tienda=df_tienda.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tienda['id_tienda'] = df_tienda['id_tienda'] + 1
df_tienda

,id_tienda,nombre_tienda,direccion,ciudad
0,1,SUPER24H_Málaga_1,"Calle, Pacífico, 121",Málaga
1,2,SUPER24H_Málaga_2,"Avenida, Libertad, 133",Málaga
2,3,SUPER24H_Málaga_3,"Avenida, Conocimiento, 129",Málaga
3,4,SUPER24H_Málaga_4,"Avenida, Costitución, 1",Málaga
4,5,SUPER24H_Málaga_5,"Calle, América, 83",Málaga
5,6,SUPER24H_Madrid_1,"Avenida, América, 109",Madrid
6,7,SUPER24H_Madrid_2,"Calle, Libertad, 66",Madrid
7,8,SUPER24H_Madrid_3,"Calle, Pacífico, 67",Madrid
8,9,SUPER24H_Madrid_4,"Avenida, Libertad, 3",Madrid
9,10,SUPER24H_Madrid_5,"Avenida, Costitución, 10",Madrid


## EMPLEADOS

In [7]:
def selec_puesto(num, puestos):
    if num < 5:
        return puestos[0]
    elif num < 10:
        return puestos[1]
    elif num < 15:
        return puestos[2]
    elif num < 19:
        return puestos[3]
    else:
        return puestos[4]

puestos = ['Cajero', 'Reponedor', 'Mozo de Almacen', 'Jefe de Seccion', 'Gerente']
nombres = ['Carolina', 'Cecilia', 'Alan', 'Kike', 'Panchi', 'Carlos', 'Carmen', 'Emilio', 'Jose Ignacio', 'Luis', 'Maria', 'Marino','Nerea','Noelia', 'Oscar', 'Pablo', 'Rafael']
apellidos = ['García', 'Martínez', 'López', 'Hernández', 'González', 'Pérez', 'Rodríguez', 'Sánchez', 'Ramírez', 'Torres', 'Flores', 'Rivera', 'Díaz', 'Cruz', 'Morales']
tiendas = df_tienda['id_tienda']
empleados = []

for tienda in tiendas:
    for num in range(20):
        nombre = f'{random.choice(nombres)} {random.choice(apellidos)} {random.choice(apellidos)}'
        puesto = selec_puesto(num, puestos)
        empleados.append([nombre, puesto, tienda])
        
df_empleados = pd.DataFrame(empleados, columns=['nombre_empleado', 'puesto', 'id_tienda'])
df_empleados=df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados

,id_empleado,nombre_empleado,puesto,id_tienda
0,1,Pablo Rivera García,Cajero,1
1,2,Rafael Díaz Rodríguez,Cajero,1
2,3,Maria Torres López,Cajero,1
3,4,Noelia Pérez González,Cajero,1
4,5,Pablo Sánchez Torres,Cajero,1
...,...,...,...,...
295,296,Jose Ignacio Flores Torres,Jefe de Seccion,15
296,297,Pablo Rodríguez López,Jefe de Seccion,15
297,298,Carolina Díaz López,Jefe de Seccion,15
298,299,Cecilia Torres Sánchez,Jefe de Seccion,15


## CATEGORIAS

In [8]:
categorias = [
    'Frutería',
    'Carnicería',
    'Pescadería',
    'Panadería y Pastelería',
    'Lácteos y Huevos',
    'Bebidas',
    'Droguería y Limpieza',
    'Conservas y Alimentos enlatados',
    'Congelados',
    'Alimentos Secos y Básicos'
]

df_categorias = pd.DataFrame(categorias, columns=['nombre_categoria'])
df_categorias=df_categorias.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categorias['id_categoria'] = df_categorias['id_categoria'] + 1
df_categorias

,id_categoria,nombre_categoria
0,1,Frutería
1,2,Carnicería
2,3,Pescadería
3,4,Panadería y Pastelería
4,5,Lácteos y Huevos
5,6,Bebidas
6,7,Droguería y Limpieza
7,8,Conservas y Alimentos enlatados
8,9,Congelados
9,10,Alimentos Secos y Básicos


## PRODUCTOS

In [9]:
productos = {
    1: ['Manzanas', 'Plátanos', 'Naranjas', 'Lechugas'],  # Frutería
    2: ['Pollo', 'Ternera', 'Cerdo', 'Cordero'],           # Carnicería
    3: ['Merluza', 'Salmón', 'Atún', 'Calamares'],         # Pescadería
    4: ['Pan integral', 'Croissants', 'Donuts', 'Baguette'], # Panadería y Pastelería
    5: ['Leche', 'Queso', 'Yogur', 'Huevos'],              # Lácteos y Huevos
    6: ['Agua', 'Refrescos', 'Vino tinto', 'Cerveza'],     # Bebidas
    7: ['Detergente', 'Esponjas', 'Desinfectante', 'Papel higiénico'], # Droguería y Limpieza
    8: ['Atún enlatado', 'Sopa de tomate', 'Maíz dulce', 'Guisantes en conserva'], # Conservas y Alimentos enlatados
    9: ['Pizza congelada', 'Helado', 'Verduras congeladas', 'Nuggets de pollo'], # Congelados
    10: ['Arroz', 'Pasta', 'Lentejas', 'Harina']          # Alimentos Secos y Básicos
}

producto_completo = []

for categoria in df_categorias['id_categoria']:
    for producto in productos[categoria]:
        precio = round(random.uniform(0.50, 150.00), 2)
        stock = random.randint(0,500)
        producto_completo.append([producto, precio, stock, categoria])
        
df_productos = pd.DataFrame(producto_completo, columns=['nombre_producto','precio','stock', 'id_categoria'])
df_productos=df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Manzanas,73.40,168,1
1,2,Plátanos,59.26,345,1
2,3,Naranjas,146.92,74,1
3,4,Lechugas,134.87,292,1
4,5,Pollo,117.54,304,2
5,6,Ternera,79.95,464,2
6,7,Cerdo,108.75,92,2
7,8,Cordero,93.84,499,2
8,9,Merluza,9.24,180,3
9,10,Salmón,49.63,443,3


## CLIENTES

In [10]:
nombres = ['Carolina', 'Cecilia', 'Alan', 'Kike', 'Panchi', 'Carlos', 'Carmen', 'Emilio', 'Jose Ignacio', 'Luis', 'Maria', 'Marino','Nerea','Noelia', 'Oscar', 'Pablo', 'Rafael']
apellidos = ['García', 'Martínez', 'López', 'Hernández', 'González', 'Pérez', 'Rodríguez', 'Sánchez', 'Ramírez', 'Torres', 'Flores', 'Rivera', 'Díaz', 'Cruz', 'Morales']
emails = ['gmail', 'yahoo', 'hotmail']
codigo_postal = [29000, 28000, 20000]

cliente = []

for num in range (2000):
    nombre = f'{random.choice(nombres)}'
    apellido = f'{random.choice(apellidos)} {random.choice(apellidos)}'
    email = f'{nombre.lower()}.{apellido.split()[0][:1].lower()}{apellido.split()[1][:1].lower()}@{random.choice(emails)}.com'
    telefono = random.randint(600, 799)*1000000+random.randint(100000, 999999)
    cp = random.choice(codigo_postal) + random.randint(1, 25)
    cliente.append([nombre, email, telefono, cp])
    
df_clientes = pd.DataFrame(cliente, columns=['nombre_producto','precio','stock', 'id_categoria'])
df_clientes=df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1
df_clientes

,id_cliente,nombre_producto,precio,stock,id_categoria
0,1,Oscar,oscar.hp@hotmail.com,738624602,20022
1,2,Jose Ignacio,jose ignacio.dr@hotmail.com,614523327,28006
2,3,Carmen,carmen.mg@gmail.com,609934252,28007
3,4,Rafael,rafael.cf@hotmail.com,700400602,20007
4,5,Emilio,emilio.rr@yahoo.com,634463990,20021
...,...,...,...,...,...
1995,1996,Nerea,nerea.gg@hotmail.com,730689006,28007
1996,1997,Noelia,noelia.mg@yahoo.com,702201463,20013
1997,1998,Jose Ignacio,jose ignacio.fr@gmail.com,694957535,28022
1998,1999,Marino,marino.pc@gmail.com,765976757,29021


## ORDENES

In [14]:
import datetime
from datetime import datetime, timedelta

clientes = df_clientes['id_cliente']
empleados = df_empleados['id_empleado']
ordenes = []

fecha_inicial = datetime(year=2017, month=1, day=2, hour=9, minute=00)
for orden in range (10000):
    cliente = int(random.choice(clientes))
    empleado = int(random.choice(empleados))
    fecha = fecha_inicial + timedelta(minutes=30*orden)
    metodo = random.choice(['Tarjeta', 'Efectivo'])
    ordenes.append([cliente, empleado, fecha, metodo])
    
df_ordenes = pd.DataFrame(ordenes, columns=['id_cliente','id_empleado', 'fecha_orden', 'metodo_pago'])
df_ordenes=df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1
df_ordenes

,id_orden,id_cliente,id_empleado,fecha_orden,metodo_pago
0,1,1547,129,2017-01-02 09:00:00,Tarjeta
1,2,1962,191,2017-01-02 09:30:00,Tarjeta
2,3,1579,62,2017-01-02 10:00:00,Efectivo
3,4,890,285,2017-01-02 10:30:00,Efectivo
4,5,933,120,2017-01-02 11:00:00,Efectivo
...,...,...,...,...,...
9995,9996,29,276,2017-07-29 14:30:00,Tarjeta
9996,9997,1967,179,2017-07-29 15:00:00,Efectivo
9997,9998,767,215,2017-07-29 15:30:00,Efectivo
9998,9999,873,127,2017-07-29 16:00:00,Efectivo


## DETALLE ORDEN

In [ ]:
productos = df_productos['id_producto']


producto = int(random.choice(productos))
descuento = float(round(random.uniform(0.00, 5.00), 2))
precio = float(round(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0]*(1-(descuento/100)),2))

## CARGA DE DATOS CON PANDAS

# PARTE 3: CONSULTAS SQL